<a href="https://colab.research.google.com/github/Bijan-K/Pytorch-Facial-Expression-Recognition/blob/main/FacialExpressionRecognitionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Facial Expression Classification/Recognition Project

In [ ]:
!touch kaggle.json
# add the key by hand then run the cells after this

In [ ]:
# Get data
!pip install -q kaggle

!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 95% 115M/121M [00:01<00:00, 78.3MB/s]
100% 121M/121M [00:01<00:00, 68.6MB/s]


In [ ]:
!pip install torchsampler

from zipfile import ZipFile

zf = ZipFile('face-expression-recognition-dataset.zip', 'r')
zf.extractall('/content')
zf.close()

In [ ]:
# Importing libraries that are needed
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
# from torch.optim import lr_scheduler
from torchsampler import ImbalancedDatasetSampler
from torchvision import datasets, models, transforms

from tqdm import tqdm
from typing import Tuple, Dict, List

In [ ]:
# checking device, setting the value for later
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# Preprocessing data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomAffine((10)),
        transforms.RandomHorizontalFlip(p=0.5),
        # transforms.RandomVerticalFlip(p=0.5),
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'validation': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

data_dir = './images/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}


train_dataloader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                              num_workers=2, sampler=ImbalancedDatasetSampler(image_datasets[x]))
                    for x in ['train']}


test_dataloader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                              num_workers=2,sampler=ImbalancedDatasetSampler(image_datasets[x]))
                  for x in ['validation']}


train_dataloader= train_dataloader["train"]
test_dataloader = test_dataloader['validation']


In [ ]:
model = torchvision.models.efficientnet_b2(progress=True)

In [ ]:
# Changing the last layer
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1408, out_features=7),
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
# training

# single step of training
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    model.train()
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

# single step of testing
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)

            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()


            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

# the root training function
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          criterion: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=criterion,
                                          optimizer=optimizer,
                                          device=device)


        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=criterion,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

In [ ]:
# Initialize training
train(model=model, train_dataloader=train_dataloader, test_dataloader=test_dataloader, optimizer=optimizer, criterion=criterion, epochs=2, device=device)

In [ ]:
# saving the Model
torch.save(model.state_dict(), './trained_model.pt')